# Exploring the Raw Titanic Dataset

Part of the journey to making a good regression model is to understand the data that we are prediciting modelling. To do this, we will perform some exploratory data analysis on the raw data from the [Titanic Kaggle Challenge](https://www.kaggle.com/c/titanic). The purpose of this challenge is to predict the probability of survival for a passsenger, given their boarding details.

<div align="center" style="width: 600px; font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://upload.wikimedia.org/wikipedia/commons/f/fd/RMS_Titanic_3.jpg"
     alt="Titanic"
     style="float: center; padding-bottom=0.5em"
     width=600px/>
The RMS Titanic
</div>

## Imports

In [1]:
import pandas as pd
import numpy as np

## Data

In [26]:
df_train = pd.read_csv('https://raw.githubusercontent.com/Explore-AI/Public-Data/master/Data/regression_sprint/titanic_train_raw.csv')
df_test = pd.read_csv('https://raw.githubusercontent.com/Explore-AI/Public-Data/master/Data/regression_sprint/titanic_test_raw.csv')

In [27]:
df_train.head()

PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S

In [28]:
df_test.head()

PassengerId  Pclass                                          Name     Sex  \
0          892       3                              Kelly, Mr. James    male   
1          893       3              Wilkes, Mrs. James (Ellen Needs)  female   
2          894       2                     Myles, Mr. Thomas Francis    male   
3          895       3                              Wirz, Mr. Albert    male   
4          896       3  Hirvonen, Mrs. Alexander (Helga E Lindqvist)  female   

    Age  SibSp  Parch   Ticket     Fare Cabin Embarked  
0  34.5      0      0   330911   7.8292   NaN        Q  
1  47.0      1      0   363272   7.0000   NaN        S  
2  62.0      0      0   240276   9.6875   NaN        Q  
3  27.0      0      0   315154   8.6625   NaN        S  
4  22.0      1      1  3101298  12.2875   NaN        S

## Questions

### Question 1

After briefly looking through the data, you may notice that some entries are missing.

Write a function that determines the percentage of missing entries for each column in the dataset. The function should return a `dict` that contains each column name as a key entry, and the percent missing entries as its value.

_**Function Specifications:**_
* Should take a pandas `DataFrame` as input and return a `dict` as output.
* The `dict` should contain a key / value pair, where the key is the column name, and the value is the percentage of missing entries in the column.
* Should be generalised to be able to work on _**ANY**_ dataframe.
* Numeric values should be rounded to two decimal places.

In [5]:
def percent_missing(df):
    out = dict()
    for column in df.columns:
        out[column] = np.round(df[column].isnull().sum() / len(df[column]) * 100, 2)
    return out

In [10]:
percent_missing(df_train)

{'PassengerId': 0.0,
 'Survived': 0.0,
 'Pclass': 0.0,
 'Name': 0.0,
 'Sex': 0.0,
 'Age': 19.87,
 'SibSp': 0.0,
 'Parch': 0.0,
 'Ticket': 0.0,
 'Fare': 0.0,
 'Cabin': 77.1,
 'Embarked': 0.22}

_**Expected Outputs:**_
```python
percent_missing(df_train) == {
    'PassengerId': 0.0,
    'Survived': 0.0,
    'Pclass': 0.0,
    'Name': 0.0,
    'Sex': 0.0,
    'Age': 19.87,
    'SibSp': 0.0,
    'Parch': 0.0,
    'Ticket': 0.0,
    'Fare': 0.0,
    'Cabin': 77.1,
    'Embarked': 0.22
}
```

### Question 2

It would be a good idea to replace some of our missing data with numerical data. Missing values can be replaced with the either the _mean_ or the _median_ of the column. Write a function that takes in as input a dataframe, column name, and a string that is either `'mean'` or `'median'`, and returns as output either the mean or median for that column.

_**Function Specifications:**_
* The function should take three inputs: `(df, column_name, choice)`, where `df` is a pandas `DataFrame`, `column_name` is a `str`, and `choice` is a `str` that defaults to `mean`.
* If the `column_name` does not exist in `df`, raise a `ValueError`.
* Should return as output the relevant value based on the `choice` input, that is, either the mean or median of that column.
* The value should be rounded to 2 decimal places.

In [12]:
def calc_mean_median(df, column_name, choice='mean'):
    if column_name not in df.columns:
        raise ValueError
    if choice == 'mean':
        return np.round(df[column_name].mean(), 2)
    elif choice == 'median':
        return np.round(df[column_name].median(), 2)

In [30]:
calc_mean_median(df_train, 'Age')

29.7

_**Expected Outputs:**_
```python
calc_mean_median(df_train, 'Age') == 29.7
calc_mean_median(df_train, 'Pclass', choice='median') == 3.0
```

### Question 3

We ultimately want to predict the survival chance of the passengers in the testing set. We can start by building a simple model using the data we already have by using _conditional probability_! Write a function that returns the survival probability of a passenger, given a variable from the dataset. 

_**Function specifications:**_
* The function should make use of the `df_train` `DataFrame` loaded earlier in this notebook.
* It should take a `column_name` as input. Assume that the column name exists in `df_train`.
* It should return a survival likelihood for each element within the given `column_name` as a number between 0 and 1. This should return a `DataFrame` that contains the chosen `column_name` as an index, and should contain a column containing the survival probabilities. You can use `groupby` and an aggragate function to help you calculate this value.

In [18]:
def survival_likelihood(column_name):
    df = df_train.groupby(by=column_name).mean()
    return df[['Survived']]

In [24]:
survival_likelihood('SibSp')

Survived
SibSp          
0      0.345395
1      0.535885
2      0.464286
3      0.250000
4      0.166667
5      0.000000
8      0.000000

_**Expected Outputs:**_
```python
survival_likelihood('Pclass')
```
> <table class="dataframe" border="1">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Survived</th>
    </tr>
    <tr>
      <th>Pclass</th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>1</th>
      <td>0.629630</td>
    </tr>
    <tr>
      <th>2</th>
      <td>0.472826</td>
    </tr>
    <tr>
      <th>3</th>
      <td>0.242363</td>
    </tr>
  </tbody>
</table>

```python
survival_likelihood('SibSp')
```
> <table class="dataframe" border="1">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Survived</th>
    </tr>
    <tr>
      <th>SibSp</th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>0.345395</td>
    </tr>
    <tr>
      <th>1</th>
      <td>0.535885</td>
    </tr>
    <tr>
      <th>2</th>
      <td>0.464286</td>
    </tr>
    <tr>
      <th>3</th>
      <td>0.250000</td>
    </tr>
    <tr>
      <th>4</th>
      <td>0.166667</td>
    </tr>
    <tr>
      <th>5</th>
      <td>0.000000</td>
    </tr>
    <tr>
      <th>8</th>
      <td>0.000000</td>
    </tr>
  </tbody>
</table>